In [1]:
import random
import networkx as nx
from matplotlib import pyplot as plt
from collections import Counter
import math
from utils.plotTools import plot_qwak
import os
import ast
import json


In [2]:
def load_list_from_file(file_path):
    with open(file_path, 'r') as file:
        data_str = file.read()
    data = [json.loads(line) for line in data_str.splitlines()]
    return data


def write_list_to_file(file_path, data):
    data_str = [str(item) for item in data]  # Convert float values to strings
    with open(file_path, 'w') as file:
        file.write('\n'.join(data_str))
        
def load_or_generate_data(file1, file2, generation_func1, generation_func2, args1=(), kwargs1={}, args2=(), kwargs2={}):
    """
    Load data from files if they exist, or generate data using specified functions.
    
    :param file1: the file path to load the first data from
    :param file2: the file path to load the second data from
    :param generation_func1: the function to generate the first data if the file doesn't exist
    :param generation_func2: the function to generate the second data if the file doesn't exist
    :param args1: tuple containing positional arguments for the first generation function
    :param kwargs1: dict containing keyword arguments for the first generation function
    :param args2: tuple containing positional arguments for the second generation function
    :param kwargs2: dict containing keyword arguments for the second generation function
    :return: a tuple containing the two datasets
    """
    
    if os.path.exists(file1) and os.path.exists(file2):
        data1 = load_list_from_file(file1)
        data2 = load_list_from_file(file2)
        print('Files exist!')
    else:
        print('Files do not exist, generating data...')
        data1 = generation_func1(*args1, **kwargs1)
        data2 = generation_func2(*args2, **kwargs2)
        
        if not os.path.exists(file1):
            write_list_to_file(file1, data1)
            
        if not os.path.exists(file2):
            write_list_to_file(file2, data2)
    
    return data1, data2

# Sisyphus gadgets

In [3]:
def create_sisyphus_wheel(n, t):
    """
    Create a Sisyphus wheel graph for a given time step.

    Parameters:
    n (int): The number of nodes in the graph.
    t (int): The time step.

    Returns:
    G (networkx.Graph): The Sisyphus wheel graph at time step t.
    """

    # Create an empty graph
    G = nx.Graph()

    # Add nodes
    G.add_nodes_from(range(n))

    # Determine the central node
    central_node = t % (n-1)  # The last node never becomes the central node

    # Add edges from all nodes to the central node
    for i in range(n):
        if i != central_node:
            G.add_edge(i, central_node)

    return G

def create_evolving_graph(nodes, T, wheel_size):
    """
    Create an evolving graph where each edge is replaced by a Sisyphus wheel.

    Parameters:
    nodes (list): The nodes of the original directed graph.
    T (int): The number of time steps.
    wheel_size (int): The size of the Sisyphus wheels that replace the edges.

    Returns:
    evolving_graph (list): A sequence of graphs representing the evolving graph.
    """

    # Create a directed graph H with the given nodes and edges that form a cycle
    H = nx.DiGraph()
    H.add_nodes_from(nodes)
    H.add_edges_from([(nodes[i], nodes[(i+1)%len(nodes)]) for i in range(len(nodes))])

    # Create an evolving graph where each edge is replaced by a Sisyphus wheel
    evolving_graph = []
    for t in range(T):
        G_t = nx.Graph()
        for u, v in H.edges:
            # Create a Sisyphus wheel for the edge (u, v)
            wheel = create_sisyphus_wheel(wheel_size, t)
            # Relabel the nodes of the wheel to match the nodes of the original graph
            wheel = nx.relabel_nodes(wheel, {i: f'{u}_{i}' for i in range(wheel_size)})
            # Add the wheel to the graph G_t
            G_t = nx.union(G_t, wheel)

        # Connect the Sisyphus wheels according to the edges in the original graph
        for u, v in H.edges:
            G_t.add_edge(f'{u}_{(t)%(wheel_size-1)}', f'{v}_{(t+1)%(wheel_size-1)}')

        evolving_graph.append(G_t)

    return evolving_graph

# Test the function
# nodes = ['u', 'v', 'w']
# T =3  # number of time steps
# wheel_size = 5  # size of the Sisyphus wheels
# evolving_graph = create_evolving_graph(nodes, T, wheel_size)

# for t, G_t in enumerate(evolving_graph):
#     plt.figure()
#     plt.title(f'Time step {t}')
#     nx.draw(G_t, with_labels=True, node_color='lightblue', node_size=1200)
# plt.show()

def random_walk_evolving(evolving_graph, start_node, num_steps=10, num_walks=1):
    """
    Perform a random walk over an evolving graph.

    Parameters:
    evolving_graph (list): A sequence of graphs representing the evolving graph.
    start_node (str): The node where the random walk starts.
    num_steps (int): The number of steps in each random walk.
    num_walks (int): The number of random walks to perform.

    Returns:
    walks (list): A list of random walks.
    """

    walks = []
    for _ in range(num_walks):
        walk = [start_node]
        for t in range(num_steps):
            # Get the graph at time step t
            graph = evolving_graph[t % len(evolving_graph)]
            # Get the neighbors of the last node in the walk
            neighbors = list(nx.neighbors(graph, walk[-1]))
            if neighbors:
                # Choose a neighbor at random
                next_node = random.choice(neighbors)
                walk.append(next_node)
            else:
                break
        walks.append(walk)
    return walks


def estimate_cover_time_evolving(evolving_graph, num_simulations=1000):
    """
    Estimate the cover time of an evolving graph.

    Parameters:
    evolving_graph (list): A sequence of graphs representing the evolving graph.
    num_simulations (int): The number of simulations to perform.

    Returns:
    cover_times (list): A list of estimated cover times for each time step.
    """

    cover_times = []

    for _ in range(num_simulations):
        total_steps = 0
        visited = set()
        start_node = random.choice(list(evolving_graph[0].nodes))
        current_node = start_node

        while len(visited) < len(evolving_graph[0].nodes):
            for t in range(len(evolving_graph)):
                # Get the graph at time step t
                graph = evolving_graph[t]
                visited.add(current_node)
                neighbors = list(nx.neighbors(graph, current_node))
                if neighbors:
                    current_node = random.choice(neighbors)
                total_steps += 1

        # Average the total steps over the number of simulations
        average_cover_time = total_steps / num_simulations
        cover_times.append(average_cover_time)

    return cover_times

In [4]:
# nodes = ['u', 'v', 'w']
# T =3  # number of time steps
# wheel_size = 5  # size of the Sisyphus wheels
# evolving_graph = create_evolving_graph2(nodes, T, wheel_size)
# dynamic_cover_time = estimate_cover_time_evolving(evolving_graph, num_simulations=5)

In [7]:
def estimate_multiple_cover_time_evolving(evolving_graphs, num_simulations=1000):
    """
    Estimate the cover time of a list of evolving graphs.

    Parameters:
    evolving_graphs (list of lists): A list of sequences of graphs representing the evolving graphs.
    num_simulations (int): The number of simulations to perform.

    Returns:
    cover_times (list): A list of estimated cover times for each evolving graph.
    """

    cover_times = []

    for evolving_graph in evolving_graphs:
        total_steps = 0

        for _ in range(num_simulations):
            visited = set()
            start_node = random.choice(list(evolving_graph[0].nodes))
            current_node = start_node

            while len(visited) < len(evolving_graph[0].nodes):
                for t in range(len(evolving_graph)):
                    # Get the graph at time step t
                    graph = evolving_graph[t]
                    visited.add(current_node)
                    neighbors = list(nx.neighbors(graph, current_node))
                    if neighbors:
                        current_node = random.choice(neighbors)
                    total_steps += 1

        # Average the total steps over the number of simulations
        average_cover_time = total_steps / num_simulations
        cover_times.append(average_cover_time)

    return cover_times

def dynamic_graph_cover_time(n_range):
    cover_times = []
    for n in n_range:
        cover_times.append(2**n)
    return cover_times

In [6]:
# nodes = ['u', 'v', 'w']
# T =3  # number of time steps
# wheel_size = 5  # size of the Sisyphus wheels
# wheel_size_range = range(2,wheel_size)
# evolving_graph = create_evolving_graph(nodes, T, wheel_size)
# evolving_graph_list = [create_evolving_graph2(nodes, T, w) for w in wheel_size_range]
# sum_range = [] 
# for graph in evolving_graph_list:
#     sum_range.append(len(graph[0]))
# print(sum_range)
# dynamic_cover_time = estimate_multiple_cover_time_evolving(evolving_graph_list, num_simulations=5)
# print(dynamic_cover_time)
# theoretical_cover_time = dynamic_graph_cover_time(sum_range)

In [5]:
# plt.plot(dynamic_cover_time)
# plt.plot(theoretical_cover_time)